In [1]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets, models
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from PIL import Image
import numpy as np
from tqdm import tqdm

In [3]:
# 구축된 .npy파일을 Pytorch DataLoader을 사용할 수 있도록 CUSTOM DATASET을 만듬.
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from skimage.transform import resize

drive.mount('/content/drive')
default_path = "/content/drive/My Drive/인공지능 수업/final/"

CUB200_TYPE_TRAIN = 1
CUB200_TYPE_TEST = 2
CUB200_TYPE_SUBMIT = 3

original_train_data = np.load(default_path + 'train_image.npy')
original_train_label = np.load(default_path + 'train_label.npy')

# Additional
# additional_train_data = np.load(default_path + 'additional_image.npy')
# additional_train_label = np.load(default_path + 'additional_label.npy')
# original_train_data = np.concatenate((original_train_data, additional_train_data), axis=0)
# original_train_label = np.concatenate((original_train_label, additional_train_label), axis=0)

# Extend
extend_train_image = np.load(default_path + 'extend_image.npy', allow_pickle=True)
extend_train_label = np.load(default_path + 'extend_label.npy', allow_pickle=True)
print(extend_train_image.shape)
print(original_train_data.shape)
original_train_data = np.concatenate((original_train_data, extend_train_image), axis=0)
original_train_label = np.concatenate((original_train_label, extend_train_label), axis=0)


train_data, test_data, train_label, test_label = train_test_split(
    original_train_data,
    original_train_label,
    test_size = 0.1,
    random_state = 1)

# train_data = original_train_data
# train_label = original_train_label


Mounted at /content/drive
(2958, 256, 256, 3)
(895, 256, 256, 3)


In [4]:
class CUB200(data.Dataset):

    def __init__(self, type, transform = None):
        super(CUB200, self).__init__()
        """
        type : int = 1, 2, 3
        """
        
        if type == CUB200_TYPE_TRAIN:
          self.image = train_data
          self.label = train_label
        elif type == CUB200_TYPE_TEST:
          self.image = test_data
          self.label = test_label
        elif type == CUB200_TYPE_SUBMIT:
          self.image = np.load(default_path + 'test_image.npy')
          self.label = np.zeros(500)
        
        self.transform = transform

    def __getitem__(self, index):
        img, target = self.image[index], self.label[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.image)

In [5]:
trainCUB = CUB200(CUB200_TYPE_TRAIN)
print(trainCUB.image.shape)
print(trainCUB.label.shape)

testCUB = CUB200(CUB200_TYPE_TEST)
print(testCUB.image.shape)
print(testCUB.label.shape)
# print(np.max(testCUB.label), np.min(testCUB.label))

submitCUB = CUB200(CUB200_TYPE_SUBMIT)
print(submitCUB.image.shape)

(3467, 256, 256, 3)
(3467,)
(386, 256, 256, 3)
(386,)
(500, 256, 256, 3)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [16]:
transform_train = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

transform_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [8]:
# CUSTOM DATASET을 이용하여 train_loader, test_loader을 구축

batch_size = 32

train_loader = torch.utils.data.DataLoader(
    dataset = CUB200(CUB200_TYPE_TRAIN, transform = transform_train),
    batch_size = batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset = CUB200(CUB200_TYPE_TEST, transform = transform_test),
    batch_size = batch_size,
    shuffle = False
)

submit_loader = torch.utils.data.DataLoader(
    dataset = CUB200(CUB200_TYPE_SUBMIT, transform = transform_test),
    batch_size = batch_size,
    shuffle = False
)

In [9]:
def training_model(model, criterion, optimizer, scheduler, num_epochs = 25):

    for epoch in range(num_epochs):
        #scheduler.step()

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 60 == 59:
                print('[%d, %5d] loss: %.7f' %
                    (epoch + 1, (i + 1), running_loss / 20))
                running_loss = 0.0
        
        # gunmo
        scheduler.step()


        train_correct = 0
        train_total = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs = inputs.squeeze()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        print('[%d epoch] Accuracy of the network on the train images: %d %%' %
              (epoch + 1, 100 * train_correct / train_total))
        
    print("End Training do it eval_accuracy")
    return model

In [10]:
def eval_accuracy(model):
    class_correct = list(0. for i in range(50))
    class_total = list(0. for i in range(50))

    correct = 0
    total = 0
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
                    
            for i in range(labels.shape[0]):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print('Accuracy of the network on test images: %d %%' % (
        100 * correct / total))            
                
    return 100 * correct / total

In [11]:
def eval_accuracy2(model1,model2,model3, model4):
    class_correct = list(0. for i in range(50))
    class_total = list(0. for i in range(50))

    correct = 0
    total = 0
    
    model1.eval()
    model2.eval()
    model3.eval()
    model4.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs1 = model1(images)
            outputs2 = model2(images)
            outputs3 = model3(images)
            outputs4 = model4(images)
            outputs = (outputs1+outputs2+outputs3+outputs4)/4
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
                    
            for i in range(labels.shape[0]):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print('Accuracy of the network on test images: %d %%' % (
        100 * correct / total))            
                
    return 

In [ ]:
class Interpolate(nn.Module):
    def __init__(self, size, mode):
        super(Interpolate, self).__init__()
        self.interp = nn.functional.interpolate
        self.size = size
        self.mode = mode
        
    def forward(self, x):
        x = self.interp(x, size=self.size, mode=self.mode, align_corners=True)
        return x

In [17]:
linear1 = nn.Linear(2048, 50, bias=True)
linear1_1 = nn.Linear(2048, 1024, bias=True)
linear1_2 = nn.Linear(2048, 50, bias=True)
linear1_3 = nn.Linear(2048, 50, bias=True)
linear2 = nn.Linear(1024, 512, bias=True)
linear3 = nn.Linear(512, 50, bias=True)
dropout = nn.Dropout(0.25)
relu = nn.ReLU()

# xavier initialization
nn.init.xavier_uniform_(linear1.weight)
nn.init.xavier_uniform_(linear1_1.weight)
nn.init.xavier_uniform_(linear1_2.weight)
nn.init.xavier_uniform_(linear1_3.weight)
nn.init.xavier_uniform_(linear2.weight)
#nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[ 0.0625, -0.0584,  0.0064,  ...,  0.0084, -0.0294, -0.0009],
        [-0.0419,  0.0343,  0.0370,  ...,  0.0604,  0.0173,  0.0264],
        [-0.0400,  0.0218, -0.0583,  ..., -0.0558, -0.0077, -0.0385],
        ...,
        [ 0.0256,  0.0181,  0.0599,  ...,  0.0426,  0.0604, -0.0218],
        [ 0.0191,  0.0469, -0.0469,  ...,  0.0105, -0.0156,  0.0511],
        [ 0.0566,  0.0193,  0.0384,  ...,  0.0499,  0.0578, -0.0499]],
       requires_grad=True)

In [18]:
#resnext101
#스케줄러: 스텝 LR
#옵티마이저: SGD, weight_decay=0
model_rsn = models.resnext101_32x8d(pretrained=True)

for param in model_rsn.parameters():
  param.requires_grad = False
#print(model_rsn)
model_rsn.fc = nn.Sequential(
        linear1
    )
#print(model_rsn)

num_epochs = 25
model_rsn.to(device)
criterion = nn.CrossEntropyLoss()

#optimizer = optim.Adam(model_rsn.parameters(), lr = 0.001)
optimizer = optim.Adam(model_rsn.parameters(), lr=0.0008, weight_decay=0)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma = 0.99)

model_rsn = training_model(model_rsn, criterion, optimizer, lr_scheduler, num_epochs)

# eval_accuracy(model_rsn)

[1,    60] loss: 11.0340208
[1 epoch] Accuracy of the network on the train images: 43 %


KeyboardInterrupt: ignored

In [ ]:
# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

# import psutil
# import humanize
# import os
# import GPUtil as GPU

In [ ]:
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
num_epochs = 16
model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.parameters(), lr = 0.0008)
# optimizer = optim.SGD(model_ft.parameters(), lr=0.0075, momentum=0.9)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 16, gamma = 0.1)
# lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,20,40], gamma= 0.1)  

model_ft = training_model(model_ft, criterion, optimizer, lr_scheduler, num_epochs)

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,    60] loss: 9.1423422
[1 epoch] Accuracy of the network on the train images: 75 %
[2,    60] loss: 3.6406914
[2 epoch] Accuracy of the network on the train images: 82 %
[3,    60] loss: 2.4515760
[3 epoch] Accuracy of the network on the train images: 86 %
[4,    60] loss: 1.8878198
[4 epoch] Accuracy of the network on the train images: 87 %
[5,    60] loss: 1.6446251
[5 epoch] Accuracy of the network on the train images: 87 %
[6,    60] loss: 1.4065498
[6 epoch] Accuracy of the network on the train images: 90 %
[7,    60] loss: 1.3054651
[7 epoch] Accuracy of the network on the train images: 90 %
[8,    60] loss: 1.0814185
[8 epoch] Accuracy of the network on the train images: 89 %
[9,    60] loss: 1.1059019
[9 epoch] Accuracy of the network on the train images: 90 %
[10,    60] loss: 0.9657002
[10 epoch] Accuracy of the network on the train images: 93 %
[11,    60] loss: 0.9279315
[11 epoch] Accuracy of the network on the train images: 92 %
[12,    60] loss: 0.7896270
[12 epoch] 

In [ ]:
eval_acc = eval_accuracy(model_ft)

Accuracy of the network on test images: 81 %


In [ ]:
import itertools

def get_result(model):
  result = []
  confidence = []

  model.eval()
  with torch.no_grad():
    for i, data in enumerate(submit_loader, 0):
      images, _ = data
      images = images.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      result.append(predicted.cpu().numpy())

      for output in outputs:
        confidence.append(torch.max(F.softmax(output, dim=0)).cpu().numpy())
  
  result = list(itertools.chain(*result))

  ca = np.array(confidence)
  # print(ca)
  confidence_count = len(np.where(ca > 0.9)[0])
  print(confidence_count)
  
  additional_train_data = []
  additional_train_label = []

  submit_image = submit_loader.dataset.image

  # print(result)

  for i in np.where(ca > 0.9)[0]:
    additional_train_data.append(submit_image[i])
    additional_train_label.append(result[i])
  
  # Remeber prev data
  import shutil
  shutil.copy(default_path + f"additional_image.npy", default_path + f"additional_library/additional_image_{confidence_count}_{eval_acc}%.npy")
  shutil.copy(default_path + f"additional_label.npy", default_path + f"additional_library/additional_label_{confidence_count}_{eval_acc}%.npy")
  
  #store new data
  np.save(default_path + "additional_image.npy", additional_train_data)
  np.save(default_path + "additional_label.npy", additional_train_label)

  return result

In [ ]:
submit_result = get_result(model_ft)

222


In [ ]:
print(submit_result)

[16, 12, 23, 23, 47, 26, 38, 24, 16, 30, 28, 47, 41, 20, 35, 26, 15, 8, 37, 7, 4, 22, 22, 32, 38, 2, 40, 36, 30, 34, 42, 37, 45, 35, 10, 33, 5, 7, 40, 48, 32, 37, 32, 7, 8, 43, 22, 24, 39, 16, 12, 25, 5, 6, 32, 48, 42, 39, 8, 45, 0, 5, 45, 12, 14, 26, 40, 49, 3, 9, 48, 11, 9, 35, 12, 20, 40, 36, 42, 35, 45, 41, 36, 26, 22, 6, 4, 48, 8, 48, 33, 44, 19, 14, 34, 9, 14, 21, 4, 27, 19, 35, 21, 23, 45, 4, 9, 44, 18, 24, 10, 47, 1, 48, 25, 11, 42, 31, 42, 35, 3, 0, 21, 29, 14, 10, 38, 18, 26, 1, 32, 35, 44, 1, 9, 33, 43, 37, 16, 2, 40, 38, 3, 38, 15, 34, 26, 12, 34, 41, 11, 6, 39, 43, 38, 0, 22, 42, 3, 38, 1, 13, 41, 15, 21, 41, 14, 10, 13, 28, 49, 44, 41, 19, 7, 27, 46, 26, 11, 13, 46, 20, 35, 42, 24, 25, 15, 42, 6, 42, 36, 27, 6, 12, 48, 25, 29, 5, 6, 26, 2, 13, 15, 29, 2, 17, 9, 14, 6, 4, 44, 38, 21, 26, 24, 21, 29, 46, 2, 27, 48, 22, 31, 12, 31, 2, 17, 18, 13, 24, 35, 8, 27, 37, 4, 18, 29, 20, 20, 8, 49, 24, 40, 13, 19, 38, 49, 6, 6, 4, 31, 34, 20, 48, 1, 42, 10, 35, 26, 10, 29, 9, 27, 47

In [ ]:
pip install pycryptodomex --no-binary :all:

     |████████████████████████████████| 3.8MB 13.7MB/s 
Skipping wheel build for pycryptodomex, due to binaries being disabled for it.
    Running setup.py install for pycryptodomex ... done


In [ ]:
import json
from base64 import b64encode
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad

def read_txt(fileName):
    with open(fileName, 'rt') as f:
        list_data = [a.strip('\n\r') for a in f.readlines()]
    return list_data

def write_json(fileName, data):
    with open(fileName, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def load_key(key_path):
    with open(key_path, "rb") as f:
        key = f.read()
    return key

def encrypt_data(key_path, ans_list, encrypt_store_path='ans.json'):
    key = load_key(key_path)
    print(key)
    data = " ".join([str(i) for i in ans_list])
    encode_data = data.encode()
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(encode_data, AES.block_size))
    iv = b64encode(cipher.iv).decode('utf-8')
    ct = b64encode(ct_bytes).decode('utf-8')
    write_json(encrypt_store_path, {'iv':iv, 'ciphertext':ct})

if __name__=="__main__":
    # 1.이메일을 통해서 전달 받은 키 파일의 경로 입력
    key_path = default_path + "team9.pem"
    # key_path = "/content/drive/My Drive/ColabNotebooks/aiproject/team9.pem"
    # 2. 예측한 결과를 텍스트 파일로 저장했을 경우 리스트로 다시 불러오기
    # 본인이 원하는 방식으로 리스트 형태로 예측 값을 불러오기만 하면 됨(순서를 지킬것)
    #raw_ans_path = "ans.txt"
    #ans = read_txt(raw_ans_path)
    #ans에 result 저장한 리스트 넣기
    ans = submit_result
    # 3. 암호화된 파일을 저장할 위치
    encrypt_ans_path = default_path + "ai_answer.json"
    # 4. 암호화!(pycrytodome 설치)
    encrypt_data(key_path, ans, encrypt_ans_path)
    print("finished!")

0 0 0 0 ... 50
...
500

np.torch([0 0 0 0 0])

